In [124]:
import numpy as np
import gc
import pprint
from scapy.all import *
from multiprocessing.pool import ThreadPool

In [125]:
stats = {'time_delta':[0,0], #[min,max,mean,normalized_mean]
         'frag':[],       
         'ttl':[],
         'header _len':[],
         'datagram_size':[],
         'dataofs':[],
         'window_size':[],
         'seq_delta':[0,0],
         'ack_delta':[0,0],
         'flags':{}
        } #flags[index,count,mean,normalized_mean]


In [141]:
def initialize(file):
    global count
    global super_stream
    count += 1
    scap  = rdpcap(file)
    pkt = scap[0]
    last_time = pkt.time
    last_seq = pkt.seq
    last_ack = pkt.ack
    stats['frag']= [pkt['IP'].frag, pkt['IP'].frag]
    stats['ttl']= [pkt['IP'].ttl,pkt['IP'].ttl]
    stats['header_len'] = [pkt['IP'].ihl,pkt['IP'].ihl]
    stats['datagram_size'] =  [pkt['IP'].len,pkt['IP'].len]
    stats['dataofs'] = [pkt['TCP'].dataofs,pkt['TCP'].dataofs]
    stats['window_size'] = [pkt['TCP'].window,pkt['TCP'].window]
    stats['flags'][pkt['IP'].flags] = 1
    #[time_delta, fragmentation_offset, ttl, header_length, datagram_length, data_offset, window_length, sequence_delta, ack_delta]
    super_stream = np.array([0,pkt['IP'].frag, pkt['IP'].ttl, pkt['IP'].ihl, pkt['IP'].len,pkt['TCP'].dataofs, pkt['TCP'].window,0,0])
    print('initial array shape ' + str(super_stream.shape))
    scap.remove(scap[0])
    for packet in range(0,len(scap)):
        count += 1
        pkt = scap[packet]
        flag = pkt['TCP'].flags
        print(str(pkt['TCP'].seq), str(pkt['TCP'].ack))
        if pkt['IP'].frag < stats['frag'][0]:
            stats['frag'][0] = pkt['IP'].frag
        if pkt['IP'].frag > stats['frag'][1]:
            stats['frag'][1] = pkt['IP'].frag
        if pkt['IP'].ttl < stats['ttl'][0]:
            stats['ttl'][0] = pkt['IP'].ttl
        if pkt['IP'].ttl > stats['ttl'][1]:
            stats['ttl'][1] = pkt['IP'].ttl
        if pkt['IP'].ihl < stats['header_len'][0]:
            stats['header_len'][0] = pkt['IP'].ihl
        if pkt['IP'].ihl > stats['header_len'][1]:
            stat['header_len'][1] = pkt['IP'].ihl
        if pkt['IP'].len < stats['datagram_size'][0]:
            stats['datagram_size'][0] = pkt['IP'].len
        if pkt['IP'].len > stats['datagram_size'][1]:
            stats['datagram_size'][1] = pkt['IP'].len
        if pkt['TCP'].dataofs < stats['dataofs'][0]:
            stats['datagram_size'][0] = pkt['TCP'].dataofs
        if pkt['TCP'].dataofs > stats['dataofs'][1]:
            stats['datagram_size'][1] = pkt['TCP'].dataofs
        if pkt['TCP'].window < stats['window_size'][0]:
            stats['window_size'][0] = pkt['TCP'].window
        if pkt['TCP'].window < stats['window_size'][0]:
            stats['window_size'][0] = pkt['TCP'].window
        try:
            stats['flags'][flag][0] += 1
        except:
            stats['flags'][flag] = [1]
        if packet == 0:
            super_stream = np.vstack([super_stream,np.array([pkt.time-last_time,pkt['IP'].frag, pkt['IP'].ttl, pkt['IP'].ihl, pkt['IP'].len, pkt['TCP'].dataofs, pkt['TCP'].window,pkt['TCP'].seq-last_seq,pkt['TCP'].ack-last_ack])])
        else:
            lst_pkt = scap[packet-1]
            super_stream = np.vstack([super_stream,np.array([pkt.time - lst_pkt.time,pkt['IP'].frag, pkt['IP'].ttl, pkt['IP'].ihl, pkt['IP'].len, pkt['TCP'].dataofs, pkt['TCP'].window,pkt['TCP'].seq - lst_pkt['TCP'].seq,pkt['TCP'].ack - lst_pkt['TCP'].ack])])
        
    
    del pkt
    del lst_pkt
    del scap
    gc.collect()
    
    super_stream = super_stream.sum(0)
    print(super_stream)

In [195]:
def describe(file):
    global count
    global super_stream
    scap = rdpcap(file)
    pprint.pprint(super_stream)
    print('new stream ' + file)
    for packet in range(0,len(scap)):
        count += 1
        pkt = scap[packet]
        flag = pkt['TCP'].flags
        if pkt['IP'].frag < stats['frag'][0]:
            stats['frag'][0] = pkt['IP'].frag
        if pkt['IP'].frag > stats['frag'][1]:
            stats['frag'][1] = pkt['IP'].frag
        if pkt['IP'].ttl < stats['ttl'][0]:
            stats['ttl'][0] = pkt['IP'].ttl
        if pkt['IP'].ttl > stats['ttl'][1]:
            stats['ttl'][1] = pkt['IP'].ttl
        if pkt['IP'].ihl < stats['header_len'][0]:
            stats['header_len'][0] = pkt['IP'].ihl
        if pkt['IP'].ihl > stats['header_len'][1]:
            stat['header_len'][1] = pkt['IP'].ihl
        if pkt['IP'].len < stats['datagram_size'][0]:
            stats['datagram_size'][0] = pkt['IP'].len
        if pkt['IP'].len > stats['datagram_size'][1]:
            stats['datagram_size'][1] = pkt['IP'].len
        if pkt['TCP'].dataofs < stats['dataofs'][0]:
            stats['datagram_size'][0] = pkt['TCP'].dataofs
        if pkt['TCP'].dataofs > stats['dataofs'][1]:
            stats['datagram_size'][1] = pkt['TCP'].dataofs
        if pkt['TCP'].window < stats['window_size'][0]:
            stats['window_size'][0] = pkt['TCP'].window
        if pkt['TCP'].window < stats['window_size'][0]:
            stats['window_size'][0] = pkt['TCP'].window
        try:
            stats['flags'][flag][0] += 1
        except:
            stats['flags'][flag] = [1]
        if packet == 0:
            
            stream = np.array([0,pkt['IP'].frag, pkt['IP'].ttl, pkt['IP'].ihl, pkt['IP'].len, pkt['TCP'].dataofs, pkt['TCP'].window,0,0])
            
#             print('stream ' + str(stream.shape))
#             pprint.pprint(stream)
            super_stream = np.vstack([super_stream,stream])
        else:
            lst_pkt = scap[packet-1]
            stream = np.array([pkt.time - lst_pkt.time,pkt['IP'].frag, pkt['IP'].ttl, pkt['IP'].ihl, pkt['IP'].len, pkt['TCP'].dataofs, pkt['TCP'].window,pkt['TCP'].seq - lst_pkt['TCP'].seq,pkt['TCP'].ack - lst_pkt['TCP'].ack])
            print('current stream')
#             print('stream ' + str(stream.shape))
#             pprint.pprint(stream)
            super_stream = np.vstack([super_stream,stream])
    
    
    del scap
    del pkt
    gc.collect()
    
    print("adding...")
    pprint.pprint(super_stream)
    super_stream =  super_stream.sum(0)
    
      

In [196]:
def calculate_mean():
    global stats
    global count
    global super_stream
    means = np.dvide(super_stream,count)
    keys = stat.keys()
    fl = keys.pop()
    for index in range(0,len(keys)):
        key = keys[index]
        stat[key].append(mean[index])
        

In [197]:
if __name__ == '__main__':
    count = 0
    indexer = 0
    super_stream = np.zeros([1,9])
    #files = os.listdir(sys.argv[1])
    files = os.listdir('dev')
    for x in range(0,len(files)):
        files[x] = 'dev/' + files[x]
    initialize(files[720])

initial array shape (9,)
103138447 0
103138447 0
[9.164866e+00 0.000000e+00 3.840000e+02 1.500000e+01 1.440000e+02
 2.100000e+01 2.457600e+04 0.000000e+00 0.000000e+00]


In [198]:
    files.remove(files[0])
    print(str(super_stream.shape))
    for file in files:
        describe(file)
#     pool = ThreadPool(6)
#     pool.map(describe,files)
#     pool.close()
#     pool.join()

(9,)
array([9.164866e+00, 0.000000e+00, 3.840000e+02, 1.500000e+01,
       1.440000e+02, 2.100000e+01, 2.457600e+04, 0.000000e+00,
       0.000000e+00])
new stream dev/49950-428-10_0_2_22-204_12_192_47.pcap
current stream
current stream
current stream
current stream
current stream
current stream
current stream
current stream
adding...
array([[ 9.16486600e+00,  0.00000000e+00,  3.84000000e+02,
         1.50000000e+01,  1.44000000e+02,  2.10000000e+01,
         2.45760000e+04,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.28000000e+02,
         5.00000000e+00,  4.80000000e+01,  7.00000000e+00,
         8.19200000e+03,  0.00000000e+00,  0.00000000e+00],
       [ 3.12638500e+00,  0.00000000e+00,  1.28000000e+02,
         5.00000000e+00,  4.80000000e+01,  7.00000000e+00,
         8.19200000e+03,  0.00000000e+00,  0.00000000e+00],
       [ 5.99688800e+00,  0.00000000e+00,  1.28000000e+02,
         5.00000000e+00,  4.80000000e+01,  7.00000000e+00,
         8.